# __Masixole Boya__
### __Data Analytics Engineer__ Technical Assessment

***
# __0. Imports__
***


##### I have imported several libraries to support various functionalities in the code:

- `os`: This library provides a way to interact with the operating system, and I use it to get the current working directory.

- `json`: It allows me to work with JSON data, and I use it to read and manipulate JSON files.


- `sqlalchemy.create_engine`: This function is from the SQLAlchemy library and is used to create a connection engine to interact with the SQL Server database.

- `sqlalchemy.text`: This function is used to represent SQL expressions as Python string objects.

- `pyodbc`: It is a Python module that makes accessing ODBC (Open Database Connectivity) databases simple.

## 0.1 Current Working Directory

##### I have set the current working directory using the `os.getcwd()` function to easily locate and read files in the file system.


In [1]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import pyodbc


## __0.1 Current Working Directory__

In [2]:
current_path = os.getcwd()
current_path

'c:\\Users\\STAFF\\Desktop\\Git_Repos\\Data_Engineering'

***
# __Question 1__: Automation - Export JSON Data and Import to SQL

***


##### I start by reading the content of the 'data.json' file using the `json.load` method. The data is then stored in the variable `json_data`.


## __1.1 Reading the json file__

In [3]:
with open(current_path+'/data.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

json_data

[{'name': 'Breanna Huffman',
  'phone': '(764) 234-7824',
  'email': 'lorem.sit@icloud.edu',
  'address': 'P.O. Box 324, 5906 In Rd.',
  'postalZip': '24-65',
  'region': 'Guerrero',
  'country': 'Philippines',
  'list': 11,
  'numberrange': 8,
  'currency': '$77.26',
  'alphanumeric': 'FYI84OMP4HD'},
 {'name': 'Nomlanga David',
  'phone': '(831) 152-6402',
  'email': 'vel.venenatis.vel@yahoo.com',
  'address': '5181 Egestas Ave',
  'postalZip': '31218',
  'region': 'Nuevo León',
  'country': 'China',
  'list': 3,
  'numberrange': 1,
  'currency': '$47.55',
  'alphanumeric': 'BYQ14QWE9PI'},
 {'name': 'Samuel Fulton',
  'phone': '(236) 478-3395',
  'email': 'ridiculus.mus.proin@outlook.couk',
  'address': 'Ap #664-2516 Magna. Av.',
  'postalZip': '745807',
  'region': 'Mexico City',
  'country': 'Turkey',
  'list': 5,
  'numberrange': 4,
  'currency': '$89.28',
  'alphanumeric': 'UFF24BGU1VD'},
 {'name': 'Kennan Macias',
  'phone': '1-987-725-3445',
  'email': 'scelerisque.dui@yahoo.com

## __1.2 Connecting to SQL Server__



##### I establish a connection to the SQL Server using the `pyodbc` library. The connection details include the driver, server name (DESKTOP-S7VHLIE\\SQLEXPRESS), the database name (CustomerTransactions), and Windows Authentication (Trusted_Connection=yes). After establishing the connection, I create a cursor to execute SQL queries. The connection is committed, and the database to be used is set to 'CustomerTransactions'.


In [4]:

conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=DESKTOP-S7VHLIE\\SQLEXPRESS;'
    'Database=CustomerTransactions;'
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()


In [5]:
conn.commit()

In [7]:
cursor.execute('use CustomerTransactions')

## __1.3 Preparing to create the SQL table__

In [8]:
columns_and_types = [(column, type(json_data[0][column]).__name__) 
                     for column in json_data[0]]
columns_and_types

[('name', 'str'),
 ('phone', 'str'),
 ('email', 'str'),
 ('address', 'str'),
 ('postalZip', 'str'),
 ('region', 'str'),
 ('country', 'str'),
 ('list', 'int'),
 ('numberrange', 'int'),
 ('currency', 'str'),
 ('alphanumeric', 'str')]

In [9]:
python_to_sql_type = {
    'str': 'NVARCHAR(MAX)',
    'int': 'INT',
    'float': 'FLOAT',
    'bool': 'BIT',
}

columns_and_types_mapped = [
    (column, python_to_sql_type[data_type.lower()]) 
    for column, data_type in columns_and_types
    ]

In [10]:
columns_and_types_mapped

[('name', 'NVARCHAR(MAX)'),
 ('phone', 'NVARCHAR(MAX)'),
 ('email', 'NVARCHAR(MAX)'),
 ('address', 'NVARCHAR(MAX)'),
 ('postalZip', 'NVARCHAR(MAX)'),
 ('region', 'NVARCHAR(MAX)'),
 ('country', 'NVARCHAR(MAX)'),
 ('list', 'INT'),
 ('numberrange', 'INT'),
 ('currency', 'NVARCHAR(MAX)'),
 ('alphanumeric', 'NVARCHAR(MAX)')]

## __1.4 Creating the SQL table__


##### I prepare to create an SQL table based on the structure of the JSON data. I extract the column names and their corresponding Python data types from the first entry of the JSON data. Using a mapping dictionary (`python_to_sql_type`), I map Python data types to their equivalent SQL Server data types. The resulting list (`columns_and_types_mapped`) contains tuples with column names and their corresponding SQL data types.


In [11]:
create_table_sql = f'''
CREATE TABLE JSON_TABLE (
    {', '.join([f'{column} {data_type}' for column, data_type in columns_and_types_mapped])}
)
'''
cursor.execute(create_table_sql)

In [12]:
conn.commit()

## __1.5 Importing JSON data into the SQL table__


##### For each record in the JSON data, I sanitize the data by replacing single quotes (`'`) with underscores (`_`) in string values. This step is essential to prevent SQL injection and ensure proper data insertion. I then construct an SQL INSERT statement based on the columns and types mapped earlier. The data values are inserted using placeholders (`?`), and the sanitized values from each record are executed using the cursor. This process ensures that the JSON data is safely and accurately inserted into the SQL table.


In [13]:
for record in json_data:
    sanitized_record = {key: value.replace("'", '_') if isinstance(value, str) else value for key, value in record.items()}

    insert_sql = f'''
    INSERT INTO JSON_TABLE ({', '.join(column for column, _ in columns_and_types_mapped)})
    VALUES ({', '.join(['?' for _ in columns_and_types_mapped])})
    '''
    cursor.execute(insert_sql, list(sanitized_record.values()))

In [14]:
conn.commit()



##### I construct an SQL SELECT statement to retrieve the top 5 rows from the SQL table named `JSON_TABLE`. The cursor executes this query, and the result is fetched and printed row by row. This step allows me to verify the successful insertion of data into the SQL table and observe the content retrieved from the table.


In [15]:
select_sql = '''
SELECT TOP 5 *
FROM JSON_TABLE
'''

result = cursor.execute(select_sql).fetchall()

for row in result:
    print(row)

('Breanna Huffman', '(764) 234-7824', 'lorem.sit@icloud.edu', 'P.O. Box 324, 5906 In Rd.', '24-65', 'Guerrero', 'Philippines', 11, 8, '$77.26', 'FYI84OMP4HD')
('Nomlanga David', '(831) 152-6402', 'vel.venenatis.vel@yahoo.com', '5181 Egestas Ave', '31218', 'Nuevo León', 'China', 3, 1, '$47.55', 'BYQ14QWE9PI')
('Samuel Fulton', '(236) 478-3395', 'ridiculus.mus.proin@outlook.couk', 'Ap #664-2516 Magna. Av.', '745807', 'Mexico City', 'Turkey', 5, 4, '$89.28', 'UFF24BGU1VD')
('Kennan Macias', '1-987-725-3445', 'scelerisque.dui@yahoo.com', '1010 Semper Rd.', '8626', 'West Region', 'Poland', 3, 2, '$3.02', 'EAJ78GOI7CG')
('Levi Fulton', '1-129-574-2583', 'nisl.arcu@protonmail.org', '997-9819 Primis Street', '100048', 'Tasmania', 'Peru', 1, 6, '$7.82', 'LWK55HMS8VM')


In [ ]:
cursor.close()
conn.close()

## __1.6 Deleting the JSON file__

In [16]:
json_file = current_path+ "/data.json"
try:
    os.remove(json_file)
    print("JSON file deleted successfully.")
except Exception as e:
    raise Exception(f"Error deleting JSON file: {str(e)}")

JSON file deleted successfully.
